# Preprocessing

In [62]:
import numpy as np
import pandas as pd
import os
import pickle
import utm


ModuleNotFoundError: No module named 'utm'

In [32]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc

In [33]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mineralogy

**To do**
* Clean last points in Excel file while using "sum" as check

In [35]:
mineralogy = pd.read_excel("../_CLEANED/Vistelius_data_cleaned.xlsx", index_col=0)

In [36]:
np.isclose(mineralogy.loc[:, :"oth"].sum(axis=1), mineralogy.loc[:, "sum"])

array([ True,  True,  True, ...,  True,  True,  True])

In [37]:
wrong_sum = mineralogy.loc[~np.isclose(mineralogy.loc[:, :"oth"].sum(axis=1), mineralogy.loc[:, "sum"]), :]

In [38]:
wrong_sum

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,l.i.,oth,sum,hs


In [39]:
mineralogy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4659 entries, 1 to 4659
Data columns (total 15 columns):
SiO2     4659 non-null float64
TiO2     4626 non-null float64
Al2O3    4659 non-null float64
Fe2O3    4657 non-null float64
FeO      4659 non-null float64
MnO      4544 non-null float64
MgO      4658 non-null float64
CaO      4659 non-null float64
Na2O     4659 non-null float64
K2O      4659 non-null float64
P2O5     3834 non-null float64
l.i.     4659 non-null float64
oth      874 non-null float64
sum      4659 non-null float64
hs       2240 non-null float64
dtypes: float64(15)
memory usage: 582.4 KB


In [40]:
wrong_sum.loc[:, :"oth"].sum(axis=1)

Series([], dtype: float64)

In [41]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,l.i.,oth,sum,hs
1,80.80,0.04,10.16,0.61,1.72,NaN,0.40,0.55,2.00,3.59,NaN,0.35,NaN,100.22,NaN
2,80.00,0.10,10.10,0.17,0.56,0.02,0.40,0.35,2.30,5.10,0.05,0.50,NaN,99.65,NaN
3,79.92,0.05,9.89,0.16,1.73,0.02,0.12,0.14,0.75,6.15,NaN,1.02,0.08,100.03,0.30
4,79.65,0.04,9.64,1.15,0.75,0.10,0.45,0.67,3.71,4.25,NaN,0.26,NaN,100.67,0.01
5,79.18,0.08,10.24,0.64,2.60,0.04,0.05,1.25,1.52,3.08,0.01,1.75,0.16,100.60,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4655,51.09,0.39,19.44,4.09,4.97,0.22,3.79,8.02,4.12,2.01,0.59,1.48,NaN,100.21,0.30
4656,50.90,1.00,17.76,2.61,8.99,0.10,5.07,8.46,2.66,1.64,0.26,0.80,NaN,100.25,NaN
4657,50.40,1.15,18.03,4.21,5.22,0.16,5.84,8.45,2.68,2.24,0.47,1.49,0.03,100.37,0.47
4658,50.34,1.13,22.01,2.28,4.18,0.10,2.73,9.41,4.06,1.09,0.46,2.60,NaN,100.39,0.47


### Cleaning
**To do**
* Replace zero values


In [42]:
x = mineralogy

In [43]:
x['SiO2'] = x['SiO2'].replace(0, 0.01)
x['TiO2'] = x['TiO2'].replace(0, 0.01)
x['Al2O3'] = x['Al2O3'].replace(0, 0.01)
x['Fe2O3'] = x['Fe2O3'].replace(0, 0.01)
x['FeO'] = x['FeO'].replace(0, 0.01)
x['MnO'] = x['MnO'].replace(0, 0.01)
x['MgO'] = x['MgO'].replace(0, 0.01)
x['CaO'] = x['CaO'].replace(0, 0.01)
x['Na2O'] = x['Na2O'].replace(0, 0.01)
x['K2O'] = x['K2O'].replace(0, 0.01)
x['P2O5'] = x['P2O5'].replace(0, 0.01)
x['l.i.'] = x['l.i.'].replace(0, 0.01)
x['oth'] = x['oth'].replace(0, 0.01)
x['hs'] = x['hs'].replace(0, 0.01)


In [44]:
x.to_excel("../_CLEANED/Vistelius_data_cleaned.xlsx")


* Normalize

In [45]:
x['SiO2'] = x['SiO2'] / x['sum'] * 100
x['TiO2'] = x['TiO2'] / x['sum'] * 100
x['Al2O3'] = x['Al2O3'] / x['sum'] * 100
x['Fe2O3'] = x['Fe2O3'] / x['sum'] * 100
x['FeO'] = x['FeO'] / x['sum'] * 100
x['MnO'] = x['MnO'] / x['sum'] * 100
x['MgO'] = x['MgO'] / x['sum'] * 100
x['CaO'] = x['CaO'] / x['sum'] * 100
x['Na2O'] = x['Na2O'] / x['sum'] * 100
x['K2O'] = x['K2O'] / x['sum'] * 100
x['P2O5'] = x['P2O5'] / x['sum'] * 100
x['l.i.'] = x['l.i.'] / x['sum'] * 100
x['oth'] = x['oth'] / x['sum'] * 100
x['sum'] = x['sum'] / x['sum'] * 100

In [46]:
x.to_excel("../_INTERPOLATION/normalised_values.xlsx")

### centred log-ratio (clr) transformation

In [47]:
mineralogy_clr = preproc.clr(mineralogy)
mineralogy_clr

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,l.i.,oth,sum,hs
1,3.705222,-3.905631,1.631703,-1.181051,-0.144431,NaN,-1.603046,-1.284592,0.006392,0.591397,NaN,-1.736577,NaN,3.920613,NaN
2,4.417642,-2.266970,2.348151,-1.736341,-0.544203,-3.876408,-0.880675,-1.014207,0.868525,1.664856,-2.960117,-0.657532,NaN,4.637279,NaN
3,4.608269,-2.768489,2.518767,-1.605339,0.775364,-3.684780,-1.893021,-1.738870,-0.060439,2.043695,NaN,0.247045,-2.298486,4.832713,-0.976430
4,4.285378,-3.311140,2.173657,0.047498,-0.379946,-2.394849,-0.890771,-0.492741,1.218768,1.354655,NaN,-1.439337,NaN,4.519584,-4.690756
5,4.525075,-2.372377,2.479653,-0.292936,1.108863,-3.065524,-2.842381,0.376495,0.572062,1.278281,-4.451819,0.712967,-1.679230,4.764504,-1.113632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4655,2.772500,-2.102697,1.806244,0.247456,0.442331,-2.675217,0.171277,0.920850,0.254764,-0.462954,-1.688722,-0.769047,NaN,3.446179,-2.362964
4656,2.704567,-1.225296,1.651652,-0.265946,0.970817,-3.527881,0.398045,0.910053,-0.246970,-0.730600,-2.572370,-1.448440,NaN,3.382371,NaN
4657,2.993652,-0.786577,1.965698,0.511123,0.726158,-2.758921,0.838392,1.207827,0.059478,-0.119863,-1.681362,-0.527563,-4.432897,3.682524,-1.677669
4658,2.786317,-1.010266,1.959014,-0.308308,0.297828,-3.435068,-0.128182,1.109290,0.268700,-1.046306,-1.909012,-0.176972,NaN,3.476579,-1.883613


### Principal Component Analysis (PCA)

##### code doesn't run with NaN values --> very small value is given to the NaN value: (0.000001)

In [48]:
mineralogy_clr['SiO2'] = mineralogy_clr['SiO2'].fillna(0.000001)
mineralogy_clr['TiO2'] = mineralogy_clr['TiO2'].fillna(0.000001)
mineralogy_clr['Al2O3'] = mineralogy_clr['Al2O3'].fillna(0.000001)
mineralogy_clr['Fe2O3'] = mineralogy_clr['Fe2O3'].fillna(0.000001)
mineralogy_clr['FeO'] = mineralogy_clr['FeO'].fillna(0.000001)
mineralogy_clr['MnO'] = mineralogy_clr['MnO'].fillna(0.000001)
mineralogy_clr['MgO'] = mineralogy_clr['MgO'].fillna(0.000001)
mineralogy_clr['Na2O'] = mineralogy_clr['Na2O'].fillna(0.000001)
mineralogy_clr['K2O'] = mineralogy_clr['K2O'].fillna(0.000001)
mineralogy_clr['P2O5'] = mineralogy_clr['P2O5'].fillna(0.000001)
mineralogy_clr['l.i.'] = mineralogy_clr['l.i.'].fillna(0.000001)
mineralogy_clr['oth'] = mineralogy_clr['oth'].fillna(0.000001)
mineralogy_clr['hs'] = mineralogy_clr['hs'].fillna(0.000001)

In [49]:
mineralogy_pca = preproc.pca(mineralogy_clr)
preproc.pca_variance(mineralogy_pca)

9 PCA components  out of 15 components with variance sum 0.9682467701388624 needed for obtaining sum of variance > 0.95


array([2.48297416e-01, 1.80014018e-01, 1.49786620e-01, 1.32862032e-01,
       8.41756392e-02, 6.96672717e-02, 5.32829776e-02, 2.68725331e-02,
       2.32882632e-02, 1.84848714e-02, 9.69142692e-03, 2.90980754e-03,
       6.40800920e-04, 2.63231061e-05, 9.96104219e-16])

In [50]:
mineralogy_pca_df = preproc.create_pca_df(mineralogy_pca, mineralogy_clr)

____

## Coordinates

In [51]:
coordinates = pd.read_excel("../_DATA/full_with_coordinates.xlsx", index_col=0, usecols=[0, 1, 2, 3])

In [52]:
coordinates

,Latitude,Longitude,past_mer
0,"44°31'30.0""","138°37'30.0""",NaN
1,"54°12'10.0""","119°24'0.0""",NaN
2,"62°36'0.0""","155°36'0.0""",NaN
3,"61°35'0.0""","146°2'0.0""",NaN
4,"68°55'0.0""","164°24'0.0""",NaN
...,...,...,...
4654,"66°42'0.0""","164°23'0.0""",NaN
4655,"46°56'30.0""","137°5'3.0""",NaN
4656,"58°12'0.0""","138°12'0.0""",NaN
4657,"60°51'0.0""","147°31'0.0""",NaN


In [53]:
# Delete negative signs in "Longitude" column for dms2dec function to work properly
coordinates["Longitude"] = coordinates["Longitude"].str.replace("-", "")

In [54]:
# Include W in "Longitude" column
sum_ = 0

for index, row in coordinates.iterrows():
    if ("W" in str(row["past_mer"])) or ("w" in str(row["past_mer"])):
        coordinates.loc[index, "Longitude"] = row["Longitude"] + "W"
        
        sum_ += 1

In [55]:
# Check that all occurences of "W" or "w" are catched
assert sum_ == int(coordinates["past_mer"].value_counts())

### Convert from degrees to decimal format

In [56]:
coordinates["Y"] = coordinates.loc[:, "Latitude"].apply(cleaning.dms2dec)
coordinates["X"] = coordinates.loc[:, "Longitude"].apply(cleaning.dms2dec)

In [57]:
# Check
coordinates.loc[42, "X"]

-178.83333333333334

In [58]:
coordinates

,Latitude,Longitude,past_mer,Y,X
0,"44°31'30.0""","138°37'30.0""",NaN,44.525000,138.625000
1,"54°12'10.0""","119°24'0.0""",NaN,54.202778,119.400000
2,"62°36'0.0""","155°36'0.0""",NaN,62.600000,155.600000
3,"61°35'0.0""","146°2'0.0""",NaN,61.583333,146.033333
4,"68°55'0.0""","164°24'0.0""",NaN,68.916667,164.400000
...,...,...,...,...,...
4654,"66°42'0.0""","164°23'0.0""",NaN,66.700000,164.383333
4655,"46°56'30.0""","137°5'3.0""",NaN,46.941667,137.084167
4656,"58°12'0.0""","138°12'0.0""",NaN,58.200000,138.200000
4657,"60°51'0.0""","147°31'0.0""",NaN,60.850000,147.516667


coordinates = coordinates.rename({"Y" : "Latitude"}, axis=1)
coordinates = coordinates.rename({"X" : "Longitude"}, axis=1)


In [59]:
coordinates

,Latitude,Longitude,past_mer,Y,X
0,"44°31'30.0""","138°37'30.0""",NaN,44.525000,138.625000
1,"54°12'10.0""","119°24'0.0""",NaN,54.202778,119.400000
2,"62°36'0.0""","155°36'0.0""",NaN,62.600000,155.600000
3,"61°35'0.0""","146°2'0.0""",NaN,61.583333,146.033333
4,"68°55'0.0""","164°24'0.0""",NaN,68.916667,164.400000
...,...,...,...,...,...
4654,"66°42'0.0""","164°23'0.0""",NaN,66.700000,164.383333
4655,"46°56'30.0""","137°5'3.0""",NaN,46.941667,137.084167
4656,"58°12'0.0""","138°12'0.0""",NaN,58.200000,138.200000
4657,"60°51'0.0""","147°31'0.0""",NaN,60.850000,147.516667


### plotting in Qgis does not work

In [60]:
coordinates.to_excel("../_COORDINATES/coordinates_decimal.xlsx")

### Convert to UTM coordinates

**To do**
* Group samples into certain groups based on spatial distribution
* Recalculate utm coordinates based on fixed zone (fixed letter and number)

In [138]:
coordinates_utm = coordinates.apply(lambda row : utm.from_latlon(row["Y"], row["X"]), axis=1)
coordinates_utm = coordinates_utm.apply(pd.Series)
coordinates_utm.columns = ["X", "Y", "ZoneNumber", "ZoneLetter"]

NameError: ("name 'utm' is not defined", 'occurred at index 0')

In [52]:
coordinates_utm

,X,Y,ZoneNumber,ZoneLetter
0,311272.566098,4.932930e+06,54,T
1,656538.925786,6.008743e+06,50,U
2,633468.918083,6.943713e+06,56,V
3,448679.859182,6.828145e+06,55,V
4,475912.972942,7.645188e+06,58,W
...,...,...,...,...
4654,472781.240413,7.398072e+06,58,W
4655,658620.016371,5.200790e+06,53,T
4656,335447.154681,6.454395e+06,54,V
4657,528075.823175,6.746190e+06,55,V


In [53]:
coordinates_utm["ZoneNumber"].value_counts()

53    971
49    802
50    689
54    655
55    279
56    232
58    174
60    151
1     139
59    133
48    132
57    130
52     90
2      40
51     39
47      3
Name: ZoneNumber, dtype: int64

In [54]:
coordinates_utm["ZoneLetter"].value_counts()

U    1875
W    1117
T     878
V     758
X      31
Name: ZoneLetter, dtype: int64

In [63]:
(coordinates_utm["ZoneNumber"].astype(str) + coordinates_utm["ZoneLetter"]).value_counts()

49U    759
50U    658
53T    643
55V    197
56V    196
54W    193
54V    178
53U    167
53W    148
1W     139
54T    129
60W    126
59W    125
54U    124
58W    111
55W     82
48U     77
58V     63
52W     58
57V     57
57W     56
48T     55
2W      40
49T     40
51U     38
56W     36
52U     32
54X     31
60V     25
50V     20
57U     17
53V     13
50T     11
59V      8
47U      3
49W      3
51V      1
dtype: int64

____

## Metadata

In [34]:
metadata = pd.read_excel("../_DATA/full_with_coordinates.xlsx", index_col=0)\
             .drop(["Latitude", "Longitude", "past_mer"], axis=1)

In [35]:
metadata

,type_granite,time,massif,sampler,others,sampler+year
0,Granite leucogranitic,K2,NaN,NaN,NaN,"V.N.Musin,1970"
1,Granite leucocratic,Tr,NaN,V.I.Zhigalova,NaN,"E.A.Ivanov,1969"
2,Granite leucocratic coarse-grained,K2,Omsukchan massif,P.M.Bosek,NaN,"O.S.Gracheva,1948"
3,Granite leucocratic,K1,Buksandzhin massif,A.Kh.Brovtman,NaN,"A.F.Mikhaylov,1948"
4,Granite-porphyry micropegmatitic,K1,Attykveem massif,L.G.Semenova,0th.:S-0.16,"A.I.Sadovsky,1963"
...,...,...,...,...,...,...
4654,Diorite,K1,Egdegkych massif,NaN,Oth.:co2-0.12,"V.A.lgnat'ev,1964"
4655,Quartz diorite,K2,Verkhneplotnikovsky massif,NaN,NaN,"A.A.Syas'ko,1969"
4656,Diorite,J3,NaN,NaN,"Oth.:co2-0.02,so3-0.0l","N.N.Remizov,1967"
4657,Diorite,K2,NaN,NaN,NaN,"A.P.Osipov,1966"


In [36]:
metadata["type_granite"].value_counts()

Granite                                    656
Granodiorite                               523
Granite-porphyry                           282
Bt granite                                 268
Granite leucocratic                        220
                                          ... 
Granodiorite leucocratic                     1
Mu granite porphyraceous coarse-grained      1
Px-Amf granodiorite                          1
Amf-Bt granite-porphyry                      1
Hb-Bt granite(granosyenite)                  1
Name: type_granite, Length: 401, dtype: int64

In [37]:
metadata["massif"].value_counts()

Ulakhan-Sis massif         58
Khoboyotuu-Echiy massif    56
Vladimirsky massif         35
Zimov'e massif             33
Bom-Gorkhon massif         31
                           ..
Shivkin massif              1
Kyragas massif              1
Dogdin massif               1
Alchan massif               1
Chekhalin massif            1
Name: massif, Length: 942, dtype: int64

In [38]:
metadata["time"].value_counts()

K2         1271
K1         1151
Tr          465
J3          411
J2          154
K2-Pg1      127
Tr-J1       122
Tr3-J1      121
J1_2        108
Tr-J         82
Pg1          78
Pg           74
J1           69
Mz           57
K            52
J            46
K2-Pg        42
J2_3         35
J3-K1        31
Mz1          28
Tr3          28
Pg1_2        20
Pg2          18
Tr2          17
K1_2          9
Tr-J2         7
Tr3-J         3
Tr2_3         3
K2_Pg         2
Tr1           2
Mz3           2
N             2
J-K           2
Tr3-J1N       2
Pg3           2
Mz1-N         1
J3-Pg         1
J2-K          1
J3-Pg1        1
K1-K2         1
J-Pg3         1
J 1_2         1
Pg2_3         1
Tr1-J1        1
Name: time, dtype: int64

In [39]:
metadata["sampler"].value_counts()

L.S.Voronova       120
D.M.Shuster         87
N.A.Lebedeva        86
V.I.Zhigalova       70
N.P.Mel'nikova      49
                  ... 
R.PKopnova           1
A.V.Skri pina        1
NP.Me l'nikova       1
N.A.Krivitskaya      1
E.N.Grigoryan        1
Name: sampler, Length: 739, dtype: int64

In [40]:
metadata["sampler+year"].value_counts()

G.A.Valuy,1975         76
V.A.Popeko,1968        61
V.A.Faradzhev,1971     43
V.S.Ivanov,1968        37
R.O.Galabala,1976      37
                       ..
I.N.Trumpe,1958         1
N.I.Tikhomirov,1938     1
A.I.Gus'kova,1954       1
M.N.Zlobin,1941         1
M.P.Krutous,1956        1
Name: sampler+year, Length: 1392, dtype: int64

## Saving of data

In [ ]:
# Save data as pickle files to use them in later notebooks
preproc.save_obj(mineralogy) # mineralogy
preproc.save_obj(mineralogy_clr) # mineralogy clr
preproc.save_obj(mineralogy_pca) # mineralogy pca info
preproc.save_obj(mineralogy_pca_df) # mineralogy pca scores

preproc.save_obj(___) # coordinates latlon
preproc.save_obj(___) # coordinates utm
preproc.save_obj(___) # metadata

____